In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
import keras.layers as kl
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from sklearn.model_selection import train_test_split


from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
data_pd = pd.read_csv('D:\DS_2nd_year\semester_2nd\[DS108]\Doan\dataset_ds108\isic_dermnet.csv')
data_pd.head()

,image_id,age_approx,anatom_site_general,dx,sex,lesion_id
0,ISIC_4695565,75.0,posterior torso,akiec,female,IL_2485486
1,ISIC_4407081,65.0,head/neck,akiec,female,IL_5173845
2,ISIC_6803155,70.0,head/neck,akiec,male,IL_8876724
3,ISIC_6037786,55.0,head/neck,akiec,female,IL_1728878
4,ISIC_4263601,75.0,head/neck,akiec,male,IL_6263448


In [3]:
train_dir_8000 = os.path.join('dataset_ds108', 'train_dir_8000')
test_dir_8000 = os.path.join('dataset_ds108', 'test_dir_8000')

In [4]:
df_count = data_pd.groupby('lesion_id').count()
df_count.head()

,image_id,age_approx,anatom_site_general,dx,sex
lesion_id,,,,,
DERMNETakiec_0,1,0,0,1,0
DERMNETakiec_1,1,0,0,1,0
DERMNETakiec_10,1,0,0,1,0
DERMNETakiec_11,1,0,0,1,0
DERMNETakiec_12,1,0,0,1,0


In [5]:
df_count = df_count[df_count['dx'] == 1]
df_count.reset_index(inplace=True)

In [6]:
def duplicates(x):
    unique = set(df_count['lesion_id'])
    if x in unique:
        return 'no' 
    else:
        return 'duplicates'

In [7]:
data_pd['is_duplicate'] = data_pd['lesion_id'].apply(duplicates)
data_pd.head()

,image_id,age_approx,anatom_site_general,dx,sex,lesion_id,is_duplicate
0,ISIC_4695565,75.0,posterior torso,akiec,female,IL_2485486,no
1,ISIC_4407081,65.0,head/neck,akiec,female,IL_5173845,no
2,ISIC_6803155,70.0,head/neck,akiec,male,IL_8876724,no
3,ISIC_6037786,55.0,head/neck,akiec,female,IL_1728878,no
4,ISIC_4263601,75.0,head/neck,akiec,male,IL_6263448,no


In [8]:
df_count = data_pd[data_pd['is_duplicate'] == 'no']

In [ ]:
def find_smallest_dimensions(folder_path):
    min_width = float('inf')
    min_height = float('inf')

    # Loop through every file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            file_path = os.path.join(folder_path, filename)
            with Image.open(file_path) as img:
                width, height = img.size
                if width < min_width:
                    min_width = width
                if height < min_height:
                    min_height = height

    return min_width, min_height

my_dir = os.path.join('dataset_ds108', 'my_dir')
os.mkdir(my_dir)

#find img

import os
from PIL import Image

def find_smallest_dimensions(folder_path):
    min_width = float('inf')
    min_height = float('inf')

    # Loop through every file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            file_path = os.path.join(folder_path, filename)
            with Image.open(file_path) as img:
                width, height = img.size
                if width < min_width:
                    min_width = width
                if height < min_height:
                    min_height = height

    return min_width, min_height

#Hair removal 

def scale_remove_hair(input_path,output_path):
    #Read image
    image=cv2.imread(input_path,cv2.IMREAD_COLOR)
    image_size = 224
    height, width = image.shape[:2]  # assuming image is a grayscale image or color image read by OpenCV
    start_y = (height - image_size) // 2
    start_x = (width - image_size) // 2
    img = image[start_y:start_y+image_size, start_x:start_x+image_size]
        
    #DULL RAZOR (REMOVE HAIR)

    #Gray scale
    grayScale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY )
    #Black hat filter
    kernel = cv2.getStructuringElement(1,(9,9)) 
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    #Gaussian filter
    bhg= cv2.GaussianBlur(blackhat,(3,3),cv2.BORDER_DEFAULT)
    #Binary thresholding (MASK)
    ret,mask = cv2.threshold(bhg,10,255,cv2.THRESH_BINARY)
    #Replace pixels of the mask
    dst = cv2.inpaint(img,mask,6,cv2.INPAINT_TELEA)   

    cv2.imwrite(output_path,dst)

dataset_hair_removed = os.path.join(my_dir, 'dataset_hair_removed')
os.makedirs(dataset_hair_removed, exist_ok=True)
for image_id in data_pd['image_id']:
    image_path = os.path.join('D:\SkinCancer\data', image_id + '.JPG')
    new_image_path = os.path.join(dataset_hair_removed, image_id + '.jpg')
    scale_remove_hair(image_path,new_image_path)

In [9]:
train, test_df = train_test_split(df_count, test_size=0.15, stratify=df_count['dx'])

c:\Users\Lenovo\anaconda3\envs\preprocess\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\Lenovo\anaconda3\envs\preprocess\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [10]:
def identify_trainOrtest(x):
    test_data = set(test_df['image_id'])
    if str(x) in test_data:
        return 'test'
    else:
        return 'train'

#creating train_df
data_pd['train_test_split'] = data_pd['image_id'].apply(identify_trainOrtest)
train_df = data_pd[data_pd['train_test_split'] == 'train']
train_df.head()

,image_id,age_approx,anatom_site_general,dx,sex,lesion_id,is_duplicate,train_test_split
0,ISIC_4695565,75.0,posterior torso,akiec,female,IL_2485486,no,train
1,ISIC_4407081,65.0,head/neck,akiec,female,IL_5173845,no,train
2,ISIC_6803155,70.0,head/neck,akiec,male,IL_8876724,no,train
4,ISIC_4263601,75.0,head/neck,akiec,male,IL_6263448,no,train
5,ISIC_6950870,70.0,head/neck,akiec,male,IL_2635206,no,train


In [11]:
test_df.head()

,image_id,age_approx,anatom_site_general,dx,sex,lesion_id,is_duplicate
4680,ISIC_7287303,65.0,lower extremity,mel,male,IL_1928313,no
8049,ISIC_4448257,35.0,NaN,nv,male,IL_7984900,no
4373,ISIC_1635231,40.0,lower extremity,mel,female,IL_9564184,no
4629,ISIC_6269166,75.0,upper extremity,mel,female,IL_4581908,no
6164,ISIC_3782782,60.0,anterior torso,nv,male,IL_0340594,no


In [12]:
# Image id of train and test images
train_list = list(train_df['image_id'])
test_list = list(test_df['image_id'])

In [13]:
len(test_list)

680

In [14]:
len(train_list)

8028

In [15]:
# Set the image_id as the index in data_pd
data_pd.set_index('image_id', inplace=True)

In [16]:
os.mkdir(train_dir_8000)
os.mkdir(test_dir_8000)

In [17]:
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [18]:
for i in targetnames:
  directory1=train_dir_8000+'/'+i
  directory2=test_dir_8000+'/'+i
  os.mkdir(directory1)
  os.mkdir(directory2)

In [19]:
for image in train_list:
    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    # path of source image 
    source = os.path.join('D:\DS_2nd_year\semester_2nd\[DS108]\Doan\dataset_ds108\images', file_name)

    # copying the image from the source to target file
    target = os.path.join(train_dir_8000, label, file_name)

    shutil.copyfile(source, target)

In [20]:
for image in test_list:

    file_name = image+'.jpg'
    label = data_pd.loc[image, 'dx']

    # path of source image 
    source = os.path.join('D:\DS_2nd_year\semester_2nd\[DS108]\Doan\dataset_ds108\images', file_name)

    # copying the image from the source to target file
    target = os.path.join(test_dir_8000, label, file_name)

    shutil.copyfile(source, target)

In [21]:
targetnames = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

# Augmenting images and storing them in temporary directories 
for img_class in targetnames:

    #creating temporary directories
    # creating a base directory
    my_aug_dir_8000 = 'my_aug_dir_8000'
    os.mkdir(my_aug_dir_8000)
    # creating a subdirectory inside the base directory for images of the same class
    img_dir_8 = os.path.join(my_aug_dir_8000, 'img_dir_8')
    os.mkdir(img_dir_8)

    img_list = os.listdir('dataset_ds108/train_dir_8000/' + img_class)

    # Copy images from the class train dir to the img_dir 
    for file_name in img_list:

        # path of source image in training directory
        source = os.path.join('dataset_ds108/train_dir_8000/' + img_class, file_name)

        # creating a target directory to send images 
        target = os.path.join(img_dir_8, file_name)

        # copying the image from the source to target file
        shutil.copyfile(source, target)

    # Temporary augumented dataset directory.
    source_path = my_aug_dir_8000

    # Augmented images will be saved to training directory
    save_path = 'dataset_ds108/train_dir_8000/' + img_class

    # Creating Image Data Generator to augment images
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(

        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )

    batch_size = 50
    aug_datagen = datagen.flow_from_directory(source_path,save_to_dir=save_path,save_format='jpg',target_size=(224, 224),batch_size=batch_size)

    # Generate the augmented images
    aug_images = 8000 

    num_files = len(os.listdir(img_dir_8))
    num_batches = int(np.ceil((aug_images - num_files) / batch_size))

    # creating 8000 augmented images per class
    for i in range(0, num_batches):
        images, labels = next(aug_datagen)

    # delete temporary directory 
    shutil.rmtree('my_aug_dir_8000')


Found 1281 images belonging to 1 classes.
Found 1444 images belonging to 1 classes.
Found 995 images belonging to 1 classes.
Found 281 images belonging to 1 classes.
Found 1054 images belonging to 1 classes.
Found 2691 images belonging to 1 classes.
Found 282 images belonging to 1 classes.


In [22]:
pip install img2vec_pytorch

Note: you may need to restart the kernel to use updated packages.


In [23]:
import pickle
from tqdm import tqdm
from img2vec_pytorch import Img2Vec

c:\Users\Lenovo\anaconda3\envs\preprocess\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [41]:
# prepare data
from PIL import Image
img2vec = Img2Vec()

data_dir = fr"D:\DS_2nd_year\semester_2nd\[DS108]\Doan\dataset_ds108"
train_dir = os.path.join(data_dir, 'train_dir_8000')
test_dir = os.path.join(data_dir, 'test_dir_8000')

data = {}
for j, dir_ in enumerate([train_dir, test_dir]):
    features = []
    labels = []
    for category in os.listdir(dir_):
        for img_path in os.listdir(os.path.join(dir_, category)):
            img_path_ = os.path.join(dir_, category, img_path)
            img = Image.open(img_path_).convert("RGB")
            #img = img.resize((224, 224))

            img_features = img2vec.get_vec(img)

            features.append(img_features)
            labels.append(category)

    data[['training_data', 'testing_data'][j]] = features
    data[['training_labels', 'testing_label'][j]] = labels

c:\Users\Lenovo\anaconda3\envs\preprocess\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\anaconda3\envs\preprocess\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [42]:
data.keys()

dict_keys(['training_data', 'training_labels', 'testing_data', 'testing_label'])

In [43]:
assert len(data['training_data']) == len(data['training_labels'])

In [44]:
np.savez('data_8000.npz', **data)

In [45]:
test_data = np.load('data_8000.npz')

In [46]:
test_data.keys()

KeysView(NpzFile 'data_8000.npz' with keys: training_data, training_labels, testing_data, testing_label)

# SVM

In [47]:
from sklearn.svm import SVC
model = SVC()
model.fit(test_data['training_data'], test_data['training_labels'])

SVC()

In [48]:
print("\nTest Classification Report:")
print(classification_report(test_data['testing_label'], model.predict(test_data['testing_data'])))

print("\nTest Confusion Matrix:")
print(confusion_matrix(test_data['testing_label'], model.predict(test_data['testing_data'])))

# Calculate and print additional metrics
test_accuracy = accuracy_score(test_data['testing_label'], model.predict(test_data['testing_data']))
print(f"\nTest Accuracy: {test_accuracy}")


Test Classification Report:
              precision    recall  f1-score   support

       akiec       0.33      0.65      0.44        40
         bcc       0.72      0.64      0.68        98
         bkl       0.36      0.24      0.29        50
          df       0.57      0.21      0.31        19
         mel       0.63      0.41      0.50       146
          nv       0.78      0.91      0.84       309
        vasc       0.68      0.72      0.70        18

    accuracy                           0.67       680
   macro avg       0.58      0.54      0.54       680
weighted avg       0.67      0.67      0.66       680


Test Confusion Matrix:
[[ 26   4   2   0   3   5   0]
 [ 21  63   5   1   5   2   1]
 [ 10   6  12   0   7  15   0]
 [  1   4   3   4   1   5   1]
 [ 15   5  11   1  60  52   2]
 [  5   3   0   1  18 280   2]
 [  0   2   0   0   1   2  13]]

Test Accuracy: 0.6735294117647059


# Random Forest Classifier

In [49]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(test_data['training_data'], test_data['training_labels'])

RandomForestClassifier(random_state=42)

In [50]:
print("\nTest Classification Report:")
print(classification_report(test_data['testing_label'], model.predict(test_data['testing_data'])))

print("\nTest Confusion Matrix:")
print(confusion_matrix(test_data['testing_label'], model.predict(test_data['testing_data'])))

# Calculate and print additional metrics
test_accuracy = accuracy_score(test_data['testing_label'], model.predict(test_data['testing_data']))
print(f"\nTest Accuracy: {test_accuracy}")


Test Classification Report:
              precision    recall  f1-score   support

       akiec       0.24      0.42      0.31        40
         bcc       0.62      0.56      0.59        98
         bkl       0.29      0.24      0.26        50
          df       0.67      0.21      0.32        19
         mel       0.58      0.27      0.37       146
          nv       0.73      0.91      0.81       309
        vasc       0.58      0.61      0.59        18

    accuracy                           0.62       680
   macro avg       0.53      0.46      0.47       680
weighted avg       0.62      0.62      0.60       680


Test Confusion Matrix:
[[ 17   7   2   0   6   8   0]
 [ 21  55   8   1   5   6   2]
 [  5  12  12   0   6  15   0]
 [  1   3   3   4   0   7   1]
 [ 19   8  12   0  40  65   2]
 [  6   3   2   1  12 282   3]
 [  1   1   2   0   0   3  11]]

Test Accuracy: 0.6191176470588236


# Light Gradient-Boost Machine 

In [38]:
pip install lightgbm

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.3 MB 1.1 MB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.3 MB 2.1 MB/s eta 0:00:01
   ------------- -------------------------- 0.5/1.3 MB 3.5 MB/s eta 0:00:01
   --------------------- ------------------ 0.7/1.3 MB 4.1 MB/s eta 0:00:01
   -------------------------------- ------- 1.1/1.3 MB 5.2 MB/s eta 0:00:01
   ---------------------------------------  1.3/1.3 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [51]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(boosting_type='gbdt', num_leaves=31, learning_rate=0.1, n_estimators=100)
model.fit(test_data['training_data'], test_data['training_labels'])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.208278 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 130560
[LightGBM] [Info] Number of data points in the train set: 54864, number of used features: 512
[LightGBM] [Info] Start training from score -1.933448
[LightGBM] [Info] Start training from score -1.922919
[LightGBM] [Info] Start training from score -1.924167
[LightGBM] [Info] Start training from score -1.982515
[LightGBM] [Info] Start training from score -1.960007
[LightGBM] [Info] Start training from score -1.921424
[LightGBM] [Info] Start training from score -1.979080


LGBMClassifier()

In [52]:
print("\nTest Classification Report:")
print(classification_report(test_data['testing_label'], model.predict(test_data['testing_data'])))

print("\nTest Confusion Matrix:")
print(confusion_matrix(test_data['testing_label'], model.predict(test_data['testing_data'])))

# Calculate and print additional metrics
test_accuracy = accuracy_score(test_data['testing_label'], model.predict(test_data['testing_data']))
print(f"\nTest Accuracy: {test_accuracy}")


Test Classification Report:
              precision    recall  f1-score   support

       akiec       0.29      0.50      0.37        40
         bcc       0.65      0.61      0.63        98
         bkl       0.31      0.24      0.27        50
          df       0.33      0.11      0.16        19
         mel       0.56      0.36      0.44       146
          nv       0.75      0.88      0.81       309
        vasc       0.65      0.61      0.63        18

    accuracy                           0.63       680
   macro avg       0.51      0.47      0.47       680
weighted avg       0.62      0.63      0.62       680


Test Confusion Matrix:
[[ 20   7   3   0   6   3   1]
 [ 17  60   8   1   6   4   2]
 [  9   8  12   0   3  18   0]
 [  3   4   2   2   2   5   1]
 [ 13   7  12   1  53  59   1]
 [  6   4   2   1  22 273   1]
 [  0   2   0   1   2   2  11]]

Test Accuracy: 0.6338235294117647


# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(test_data['training_data'], test_data['training_labels'])

In [ ]:
print("\nTest Classification Report:")
print(classification_report(test_data['testing_label'], model.predict(test_data['testing_data'])))

print("\nTest Confusion Matrix:")
print(confusion_matrix(test_data['testing_label'], model.predict(test_data['testing_data'])))

# Calculate and print additional metrics
test_accuracy = accuracy_score(test_data['testing_label'], model.predict(test_data['testing_data']))
print(f"\nTest Accuracy: {test_accuracy}")